In [5]:
import requests
import pandas as pd

def fetch_smartsheet_data(sheet_id):
    """
    Fetches Smartsheet data and returns it as a Pandas DataFrame.
    
    Parameters:
    sheet_id (str): The ID of the Smartsheet to fetch data from.
    bearer_token (str): The Smartsheet API bearer token for authorization.
    
    Returns:
    pd.DataFrame: DataFrame containing the sheet's data.
    """
    
    bearer_token = 'hhQ21NI9mu4JzKmskRS19wfiY7lX4smNGDUAo'
    
    headers = {
        'Authorization': f'Bearer {bearer_token}'
    }
    

    url = f'https://api.smartsheet.com/2.0/sheets/{sheet_id}'
    
    response = requests.get(url, headers=headers)
    sheet_data = response.json()
    
    columns = {col['id']: col['title'] for col in sheet_data['columns']}
    data = []

    
    for row in sheet_data['rows']:
        row_data = {columns[cell['columnId']]: cell.get('value', None) for cell in row['cells']}
        data.append(row_data)
    

    df = pd.DataFrame(data)
    return df


In [6]:
def fetch_all_sheets(bearer_token):
    """
    Fetches all available sheets and returns their data in a dictionary of DataFrames.
    
    Parameters:
    bearer_token (str): The Smartsheet API bearer token for authorization.
    
    Returns:
    dict: A dictionary where each key is the sheet name, and each value is a DataFrame containing the sheet's data.
    """
    headers = {
        'Authorization': f'Bearer {bearer_token}'
    }
    
    url = 'https://api.smartsheet.com/2.0/sheets'
    response = requests.get(url, headers=headers)
    sheets_data = response.json()['data']
    
    all_sheets_data = {}
    
    for sheet_info in sheets_data:
        sheet_id = sheet_info['id']
        sheet_name = sheet_info['name']
        print(f"Fetching data for sheet: {sheet_name} (ID: {sheet_id})")
        
        df = fetch_smartsheet_data(sheet_id)
        all_sheets_data[sheet_name] = df
    
    return all_sheets_data

bearer_token = 'hhQ21NI9mu4JzKmskRS19wfiY7lX4smNGDUAo'
all_sheets = fetch_all_sheets(bearer_token)

for sheet_name, df in all_sheets.items():
    print(f"\nData for sheet '{sheet_name}':")
    print(df)


Fetching data for sheet: (Archived) SAN ANTONIO, Y8, 5/1/23 - 4/30/24 (ID: 6100314079842180)
Fetching data for sheet: (Clinicians Only) Clinic Staff Turnover - 2024 (ID: 3474483830542212)
Fetching data for sheet: (Clinicians Only) Clinic Turnover 2024 - SAN (ID: 4906779712376708)
Fetching data for sheet: Clinic Staff Turnover - 2024 (ID: 5216127428808580)
Fetching data for sheet: Clinic Staff Turnover - 2024 (minus I&F) (ID: 133574799871876)
Fetching data for sheet: Clinic Turnover 2024 (minus I&F) - SAN (ID: 3402598795136900)
Fetching data for sheet: Clinic Turnover 2024 - SAN (ID: 2246047472504708)
Fetching data for sheet: Clinic Turnover GY8 (minus I&F) - SAN (ID: 8634708246089604)
Fetching data for sheet: Clinic Turnover GY8 - SAN (ID: 5153448274120580)
Fetching data for sheet: STAFF DIRECTORY - SAN (ID: 7870618436691844)
Fetching data for sheet: Staffing Ratio - 2021+ (ID: 2280819810166660)

Data for sheet '(Archived) SAN ANTONIO, Y8, 5/1/23 - 4/30/24':
                   Cohen Cl

In [9]:
def fetch_sheet_by_name(sheet_name, bearer_token):
    """
    Fetches data for a specific sheet by its name.
    
    Parameters:
    sheet_name (str): The name of the sheet to fetch.
    bearer_token (str): The Smartsheet API bearer token for authorization.
    
    Returns:
    pd.DataFrame or None: DataFrame containing the sheet's data if found, otherwise None.
    """
    headers = {
        'Authorization': f'Bearer {bearer_token}'
    }
    
    url = 'https://api.smartsheet.com/2.0/sheets'
    response = requests.get(url, headers=headers)
    sheets_data = response.json()['data']
    
    sheet_info = next((sheet for sheet in sheets_data if sheet['name'] == sheet_name), None)
    
    if sheet_info:
        return fetch_smartsheet_data(sheet_info['id'])
    else:
        print(f"Sheet '{sheet_name}' not found.")
        return None


bearer_token = 'hhQ21NI9mu4JzKmskRS19wfiY7lX4smNGDUAo'
sheet_name = "(Clinicians Only) Clinic Staff Turnover - 2024"
df = fetch_sheet_by_name(sheet_name, bearer_token)
print(df)

Found sheet '(Clinicians Only) Clinic Staff Turnover - 2024' with ID: 3474483830542212
               Cohen Clinic  # Separated Employees  Avg. # Employees  Turnover
0               Addison, TX                    1.0          8.083333  0.123711
1   Anchorage/Fairbanks, AK                    2.0          8.666667  0.230769
2           Clarksville, TN                    0.0          8.916667  0.000000
3      Colorado Springs, CO                    0.0          4.166667  0.000000
4               El Paso, TX                    3.0          9.250000  0.324324
5          Fayetteville, NC                    1.0         10.000000  0.100000
6            Hinesville, GA                    0.0          4.250000  0.000000
7          Jacksonville, FL                    0.0          6.000000  0.000000
8          Jacksonville, NC                    1.0          8.583333  0.116505
9               Killeen, TX                    3.0          5.166667  0.580645
10             Lakewood, WA                 